In [19]:
# Import libraries
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from scipy .stats import norm 
import seaborn as sns
from matplotlib.pyplot import figure
from pylab import cm
import matplotlib.font_manager as fm
import matplotlib as mpl
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.patches import Polygon
import matplotlib.ticker as mtick
from sklearn.metrics import r2_score
from itertools import combinations
import seaborn as sns
from matplotlib.patches import Patch, Polygon
import os
import pandas as pd
import itertools

In [20]:
DATA_SOURCE = "./crispy_3419_companies_bu.csv"
output_folder = "./square_quadrants"

print(pd.read_csv(DATA_SOURCE)["scenario_duo"].unique() )

# unique scenario duos
print(pd.read_csv(DATA_SOURCE)["scenario_duo_bckp"].unique() )

print(pd.read_csv(DATA_SOURCE)["target_duo"].unique() )


['IPR2021_FPS' 'IPR2021_RPS' 'NGFS2021_GCAM_B2DS' 'NGFS2021_GCAM_DN0'
 'NGFS2021_GCAM_DT' 'NGFS2021_GCAM_NZ2050' 'NGFS2021_MESSAGE_B2DS'
 'NGFS2021_MESSAGE_DN0' 'NGFS2021_MESSAGE_DT' 'NGFS2021_MESSAGE_NZ2050'
 'NGFS2021_REMIND_B2DS' 'NGFS2021_REMIND_DN0' 'NGFS2021_REMIND_DT'
 'NGFS2021_REMIND_NZ2050' 'Oxford2021_fast' 'WEO2021_NZE_2050'
 'WEO2021_SDS']
['IPR2021_baseline&IPR2021_FPS' 'IPR2021_baseline&IPR2021_RPS'
 'NGFS2021_GCAM_NDC&NGFS2021_GCAM_B2DS'
 'NGFS2021_GCAM_NDC&NGFS2021_GCAM_DN0'
 'NGFS2021_GCAM_NDC&NGFS2021_GCAM_DT'
 'NGFS2021_GCAM_NDC&NGFS2021_GCAM_NZ2050'
 'NGFS2021_MESSAGE_NDC&NGFS2021_MESSAGE_B2DS'
 'NGFS2021_MESSAGE_NDC&NGFS2021_MESSAGE_DN0'
 'NGFS2021_MESSAGE_NDC&NGFS2021_MESSAGE_DT'
 'NGFS2021_MESSAGE_NDC&NGFS2021_MESSAGE_NZ2050'
 'NGFS2021_REMIND_NDC&NGFS2021_REMIND_B2DS'
 'NGFS2021_REMIND_NDC&NGFS2021_REMIND_DN0'
 'NGFS2021_REMIND_NDC&NGFS2021_REMIND_DT'
 'NGFS2021_REMIND_NDC&NGFS2021_REMIND_NZ2050'
 'Oxford2021_base&Oxford2021_fast' 'WEO2021_STEPS&WEO2021_NZE_205

    # Merge the calculated values back into the original DataFrame using 'company_name' as the key
    data = data.merge(gcam_df[['company_name', 'GCAM_company_value_changes', 'GCAM_pd_difference']], on=['company_name', "business_unit"], how='left')


In [21]:
def make_data():
    return pd.read_csv(DATA_SOURCE)


def prep_data(filtered_data, x_scenario_duo, y_scenario_duo):
    ##B2DS run
    data=filtered_data
    
    # Filter the DataFrame to select only rows where 'scenario_duo' is 'NGFS2021_GCAM_NDC&NGFS2021_GCAM_B2DS'
    gcam_df = data.loc[data['combined_duo'] == x_scenario_duo,:].copy()
    
    # Calculate 'GCAMROC' as 'net_present_value_rate_of_change' for the filtered rows
    gcam_df['GCAM_company_value_changes'] = gcam_df['net_present_value_rate_of_change']
    
    # Calculate 'GCAMpddiff' as 'pd_difference' for the filtered rows
    gcam_df['GCAM_pd_difference'] = gcam_df['pd_difference']
    
    # Merge the calculated values back into the original DataFrame using 'company_name' as the key
    data = data.merge(gcam_df[['company_name', "business_unit", 'GCAM_company_value_changes', 'GCAM_pd_difference']], on=['company_name', "business_unit"], how='left')
    
    # Filter the DataFrame to keep only the specified scenario duos
    data = data[data['combined_duo'] == y_scenario_duo]
    
    return data


In [22]:
def make_npv_quadrants_plot(data, xlab_scenario, ylab_scenario):
    ###########Adjust Plots with 45°C line
    ######################################################################################################################################################
    # Comparison value change ##Total
    ######################################################################################################################################################
    plt.style.use(['default'])
    
    #equity_company_value_changes_standard_nodup_reordered = equity_company_value_changes_standard_nodup_reordered[
    #    equity_company_value_changes_standard_nodup_reordered['sector'] == 'Power'
    #]
    
    plt.figure(figsize=(10, 10), dpi=250) # CHANGE FIRST TWO NUMBERS TO CHANGE FONT SIZE AND DIMENSION OF CHART
        
    
    plt.xlabel(xlab_scenario, fontsize=24) 
    plt.ylabel(ylab_scenario, fontsize = 24)
    plt.xlim(-1,1) # CHANGE HERE TO ZOOM ON DIFFERENT QUADRANTS
    plt.ylim(-1,4) # CHANGE HERE TO ZOOM ON DIFFERENT QUADRANTS
    plt.hlines(y=0, xmin=-90, xmax=90, linewidth=1, color="grey")
    plt.vlines(x=0, ymin=-90, ymax=180, linewidth=1, color="grey")
    
    # Define the coordinates for each quadrant
    quadrant1_coords = [[-1, -1], [-1, 0], [0, 0], [0, -1]]
    quadrant2_coords = [[-1, 0], [-1, 4], [0, 4], [0, 0]]
    quadrant3_coords = [[0, 0], [0, 4], [1, 4], [1, 0]]
    quadrant4_coords = [[0, -1], [0, 0], [1, 0], [1, -1]]
    
    # Create polygons for each quadrant with desired colors
    quadrant1_poly = Polygon(quadrant1_coords, facecolor='lightcoral', alpha=0.3)
    quadrant2_poly = Polygon(quadrant2_coords, facecolor='lightgreen', alpha=0.3)
    quadrant3_poly = Polygon(quadrant3_coords, facecolor='lightblue', alpha=0.3)
    quadrant4_poly = Polygon(quadrant4_coords, facecolor='lightyellow', alpha=0.3)
    
    # Add the polygons to the plot
    ax = plt.gca()
    ax.add_patch(quadrant1_poly)
    ax.add_patch(quadrant2_poly)
    ax.add_patch(quadrant3_poly)
    ax.add_patch(quadrant4_poly)
    #plt.scatter(equity_company_value_changes_standard_nodup_reordered[equity_company_value_changes_standard_nodup_reordered['shock_scenario']==selected_scenario]['GCAM_company_value_changes'], equity_company_value_changes_standard_nodup_reordered[equity_company_value_changes_standard_nodup_reordered['shock_scenario']==selected_scenario]['OtherScenarios_company_value_changes'],s = 8, c=equity_company_value_changes_standard_nodup_reordered[equity_company_value_changes_standard_nodup_reordered['shock_scenario']==selected_scenario]['shock_scenario'].map(colors), label=equity_company_value_changes_standard_nodup_reordered[equity_company_value_changes_standard_nodup_reordered['shock_scenario']==selected_scenario]['shock_scenario'])
    
    plt.scatter(data[['GCAM_company_value_changes']], 
                data[['net_present_value_rate_of_change']],
                s = 8, 
                c="grey")
    
    plt.title("Company Value Change (%)", fontsize = 24)
    #plt.legend(handles=legend_elements)
    plt.xticks(plt.xticks()[0], ['{:,.0%}'.format(x) for x in plt.xticks()[0]], fontsize = 18)
    plt.yticks(plt.yticks()[0], ['{:,.0%}'.format(x) for x in plt.yticks()[0]], fontsize = 18)
    
    
    # Add a 45° line
    plt.plot([-1, 1], [-1, 1], linestyle='dashed', color=(0,0,0,1), linewidth = 2.5)
    
    plt.tight_layout()
    return plt.gcf(), plt.gca()

In [23]:
def make_pd_quadrants_plot(data, xlab_scenario,  ylab_scenario):
    ###########Adjuste Plots with 45°C line
    ######################################################################################################################################################
    # Comparison PD Change per scenario 
    ######################################################################################################################################################
    plt.style.use(['default'])
    #plt.rcParams['font.family'] = 'Arial'
    
    plt.figure(figsize=(10, 10), dpi=250) # CHANGE FIRST TWO NUMBERS TO CHANGE FONT SIZE AND DIMENSION OF CHART
        
    
    plt.xlabel(xlab_scenario, fontsize=24) 
    plt.ylabel(ylab_scenario, fontsize = 24)
    plt.xlim(-0.4,0.8) # CHANGE HERE TO ZOOM ON DIFFERENT QUADRANTS
    plt.ylim(-0.4,0.8) # CHANGE HERE TO ZOOM ON DIFFERENT QUADRANTS
    plt.hlines(y=0, xmin=-90, xmax=90, linewidth=1, color="grey")
    plt.vlines(x=0, ymin=-90, ymax=180, linewidth=1, color="grey")
    
    # Define the coordinates for each quadrant
    quadrant1_coords = [[-1, -1], [-1, 0], [0, 0], [0, -1]]
    quadrant2_coords = [[-1, 0], [-1, 4], [0, 4], [0, 0]]
    quadrant3_coords = [[0, 0], [0, 4], [1, 4], [1, 0]]
    quadrant4_coords = [[0, -1], [0, 0], [1, 0], [1, -1]]
    
    # Create polygons for each quadrant with desired colors
    quadrant1_poly = Polygon(quadrant1_coords, facecolor='lightcoral', alpha=0.3)
    quadrant2_poly = Polygon(quadrant2_coords, facecolor='lightgreen', alpha=0.3)
    quadrant3_poly = Polygon(quadrant3_coords, facecolor='lightblue', alpha=0.3)
    quadrant4_poly = Polygon(quadrant4_coords, facecolor='lightyellow', alpha=0.3)
    
    # Add the polygons to the plot
    ax = plt.gca()
    ax.add_patch(quadrant1_poly)
    ax.add_patch(quadrant2_poly)
    ax.add_patch(quadrant3_poly)
    ax.add_patch(quadrant4_poly)
    
    #plt.scatter(equity_company_value_changes_standard_nodup_reordered[equity_company_value_changes_standard_nodup_reordered['shock_scenario']==selected_scenario]['GCAM_pd_difference'], equity_company_value_changes_standard_nodup_reordered[equity_company_value_changes_standard_nodup_reordered['shock_scenario']==selected_scenario]['OtherScenarios_pd_difference'],s = 8, c=equity_company_value_changes_standard_nodup_reordered[equity_company_value_changes_standard_nodup_reordered['shock_scenario']==selected_scenario]['shock_scenario'].map(colors), label=equity_company_value_changes_standard_nodup_reordered[equity_company_value_changes_standard_nodup_reordered['shock_scenario']==selected_scenario]['shock_scenario'])
    plt.scatter(data[['GCAM_pd_difference']], 
                data[['pd_difference']],
                s = 8, 
                c="grey")
    
    plt.title("Company PD Difference", fontsize = 24)
    #plt.legend(handles=legend_elements)
    plt.xticks(plt.xticks()[0], ['{:,.0%}'.format(x) for x in plt.xticks()[0]], fontsize = 18)
    plt.yticks(plt.yticks()[0], ['{:,.0%}'.format(x) for x in plt.yticks()[0]], fontsize = 18)
    
    
    # Add a 45° line
    plt.plot([-1, 1], [-1, 1], linestyle='dashed', color=(0,0,0,1), linewidth = 2.5)
    
    plt.tight_layout()
    return plt.gcf(), plt.gca()

In [24]:
def filter_data(data):
    # all_crispy <- all_crispy %>% rename[company_name_hash=company_name)%>%inner_join[mpr) %>% inner_join[df_wide%>%distinct[company_name))
    nz_duos = [
        "IPR2021_baseline&IPR2021_RPS",
        "NGFS2021_REMIND_CP&NGFS2021_REMIND_NZ2050",
        "NGFS2021_REMIND_NDC&NGFS2021_REMIND_NZ2050",
        "NGFS2021_MESSAGE_CP&NGFS2021_MESSAGE_NZ2050",
        "NGFS2021_MESSAGE_NDC&NGFS2021_MESSAGE_NZ2050",
        "NGFS2021_GCAM_NDC&NGFS2021_GCAM_NZ2050",
        "NGFS2021_GCAM_CP&NGFS2021_GCAM_NZ2050",
        "WEO2021_APS&WEO2021_NZE_2050",
        "WEO2021_STEPS&WEO2021_NZE_2050" # stated policy scenario == current policies ?
    ]
    
    dt_duos = [
        "NGFS2021_REMIND_NDC&NGFS2021_REMIND_DT",
        "NGFS2021_REMIND_CP&NGFS2021_REMIND_DT",
        "NGFS2021_MESSAGE_NDC&NGFS2021_MESSAGE_DT",
        "NGFS2021_MESSAGE_CP&NGFS2021_MESSAGE_DT",
        "NGFS2021_GCAM_NDC&NGFS2021_GCAM_DT",
        "NGFS2021_GCAM_CP&NGFS2021_GCAM_DT"
    ]
    
    dn0_duos = [
        "NGFS2021_REMIND_NDC&NGFS2021_REMIND_DN0",
        "NGFS2021_REMIND_CP&NGFS2021_REMIND_DN0",
        "NGFS2021_MESSAGE_NDC&NGFS2021_MESSAGE_DN0",
        "NGFS2021_MESSAGE_CP&NGFS2021_MESSAGE_DN0",
        "NGFS2021_GCAM_NDC&NGFS2021_GCAM_DN0",
        "NGFS2021_GCAM_CP&NGFS2021_GCAM_DN0"
    ]
    
    b2ds_duos = [
        "IPR2021_baseline&IPR2021_FPS",
        "NGFS2021_REMIND_NDC&NGFS2021_REMIND_B2DS",
        "NGFS2021_REMIND_CP&NGFS2021_REMIND_B2DS",
        "NGFS2021_MESSAGE_NDC&NGFS2021_MESSAGE_B2DS",
        "NGFS2021_MESSAGE_CP&NGFS2021_MESSAGE_B2DS",
        "NGFS2021_GCAM_NDC&NGFS2021_GCAM_B2DS",
        "NGFS2021_GCAM_CP&NGFS2021_GCAM_B2DS",
        "WEO2021_APS&WEO2021_SDS",
        "WEO2021_STEPS&WEO2021_SDS",
        "Oxford2021_base&Oxford2021_fast"
    ]
    
    # # Define conditions
    # conditions = [
    #     data['scenario_duo'].isin(nz_duos),
    #     data['scenario_duo'].isin(dt_duos),
    #     data['scenario_duo'].isin(dn0_duos),
    #     data['scenario_duo'].isin(b2ds_duos)
    # ]
    
    # # Define choices corresponding to conditions
    # choices = ['NZ2050', 'DT', 'DN0', 'B2DS']
    
    # # Use numpy.select to apply these choices based on conditions, with a default value
    # data['target_duo'] = np.select(conditions, choices, default='other')
    
    
    
    remind_duos =[
        "NGFS2021_REMIND_NDC&NGFS2021_REMIND_NZ2050",
        "NGFS2021_REMIND_NDC&NGFS2021_REMIND_DT",
        "NGFS2021_REMIND_NDC&NGFS2021_REMIND_DN0",
        "NGFS2021_REMIND_NDC&NGFS2021_REMIND_B2DS"
        # "NGFS2021_REMIND_CP&NGFS2021_REMIND_NZ2050",
        # "NGFS2021_REMIND_CP&NGFS2021_REMIND_DT",
        # "NGFS2021_REMIND_CP&NGFS2021_REMIND_DN0",
        # "NGFS2021_REMIND_CP&NGFS2021_REMIND_B2DS"
    ]
    message_duos = [
        "NGFS2021_MESSAGE_NDC&NGFS2021_MESSAGE_NZ2050",
        "NGFS2021_MESSAGE_NDC&NGFS2021_MESSAGE_DT",
        "NGFS2021_MESSAGE_NDC&NGFS2021_MESSAGE_DN0",
        "NGFS2021_MESSAGE_NDC&NGFS2021_MESSAGE_B2DS"
        # "NGFS2021_MESSAGE_CP&NGFS2021_MESSAGE_NZ2050",
        # "NGFS2021_MESSAGE_CP&NGFS2021_MESSAGE_DT",
        # "NGFS2021_MESSAGE_CP&NGFS2021_MESSAGE_DN0",
        # "NGFS2021_MESSAGE_CP&NGFS2021_MESSAGE_B2DS"
    ]
    
    gcam_duos = [
        "NGFS2021_GCAM_NDC&NGFS2021_GCAM_NZ2050",
        "NGFS2021_GCAM_NDC&NGFS2021_GCAM_DN0",
        "NGFS2021_GCAM_NDC&NGFS2021_GCAM_DT",
        "NGFS2021_GCAM_NDC&NGFS2021_GCAM_B2DS"
        # "NGFS2021_GCAM_CP&NGFS2021_GCAM_NZ2050",
        # "NGFS2021_GCAM_CP&NGFS2021_GCAM_DN0",
        # "NGFS2021_GCAM_CP&NGFS2021_GCAM_DT",
        # "NGFS2021_GCAM_CP&NGFS2021_GCAM_B2DS"
    ]
    
    iea_duos =[# stated policy scenario == current policies ?
      "WEO2021_STEPS&WEO2021_NZE_2050",
      "WEO2021_STEPS&WEO2021_SDS"]
    
    ipr_duos = [
      "IPR2021_baseline&IPR2021_RPS",
                  "IPR2021_baseline&IPR2021_FPS"
    ]
    
    
    oxford_duos = ["Oxford2021_base&Oxford2021_fast"]
    
    
    
    # Combine all duos into a single list
    use_duos = remind_duos + message_duos + gcam_duos + iea_duos + ipr_duos #+ oxford_duos
    
    # # Define conditions for the new column 'scenario_provider'
    # conditions = [
    #     data['scenario_duo'].isin(remind_duos),
    #     data['scenario_duo'].isin(message_duos),
    #     data['scenario_duo'].isin(gcam_duos),
    #     data['scenario_duo'].isin(iea_duos),
    #     data['scenario_duo'].isin(ipr_duos),
    #     data['scenario_duo'].isin(oxford_duos)
    # ]
    
    # # Corresponding choices for the conditions
    # choices = ['REMIND', 'MESSAGE', 'GCAM', 'WEO', 'IPR', 'OXFORD']
    
    # # Apply conditions and choices to create new column
    # data['scenario_provider'] = np.select(conditions, choices, default=np.nan)
    
    # Filter the DataFrame based on additional conditions
    filtered_data = data[
        (data['scenario_duo_bckp'].isin(use_duos)) &
        (data['term'] == 5) &
        (data['sector'] == "Power")
    ]
    return filtered_data

In [25]:
raw_data = make_data()
filtered_data = filter_data(raw_data)

In [26]:
import pandas as pd
import itertools

# Assuming 'data' is your DataFrame and it has been loaded correctly
# Ensure 'data' DataFrame has 'scenario_duo', 'target_duo', and 'scenario_provider' columns

# First, combine 'scenario_duo' and 'target_duo' into a single column
filtered_data['combined_duo'] = filtered_data['scenario_duo'] + "&" + filtered_data['target_duo']

# Retrieve unique combinations of 'combined_duo' and 'scenario_provider'
unique_scens = filtered_data[['combined_duo', 'scenario_provider', "target_duo"]].drop_duplicates()

# Generate unique combinations of all rows in the dataframe using their indices
combinations = list(itertools.combinations(unique_scens.index, 2))  # Pair indices, not rows directly

# Create an empty list to store the scenario pairs
scenarios_xy = []

# Loop through each combination of indices
for (idx1, idx2) in combinations:
    row1 = unique_scens.loc[idx1]
    row2 = unique_scens.loc[idx2]
    # Create a tuple with the format: (combined_duo1, scenario_provider1, combined_duo2, scenario_provider2)
    scenarios_xy.append((
        row1['combined_duo'], 
        row1['scenario_provider'] + " " +row1['target_duo'],
        row2['combined_duo'], 
        row2['scenario_provider'] + " " +row2['target_duo'],
    ))

# Now, scenarios_xy contains all combinations of combined_duo and scenario_provider from different rows


/var/folders/df/zghzv05d7xb8t9xy7y5ld_h40000gn/T/ipykernel_92465/1007469548.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['combined_duo'] = filtered_data['scenario_duo'] + "&" + filtered_data['target_duo']


In [27]:
import os
import progressbar
import matplotlib.pyplot as plt

# Define the output folder and ensure it exists
os.makedirs(output_folder, exist_ok=True) 

# Initialize the progress bar with the total count of scenarios
pbar = progressbar.ProgressBar(maxval=len(scenarios_xy)).start()

# Loop through each scenario combination
for i, (scenario_x, Scenario_x, scenario_y, Scenario_y) in enumerate(scenarios_xy):
    # Prepare the data
    data = prep_data(filtered_data, scenario_x, scenario_y)
    
    # Generate and save the PD quadrants plot
    fig, ax = make_pd_quadrants_plot(data, Scenario_x, Scenario_y)
    fig.savefig(os.path.join(output_folder, f"pd_{scenario_x}___{scenario_y}.jpg"))
    plt.close(fig)
    # Generate and save the NPV quadrants plot
    fig, ax = make_npv_quadrants_plot(data, Scenario_x, Scenario_y)
    fig.savefig(os.path.join(output_folder, f"npv_{scenario_x}___{scenario_y}.jpg"))    
    
    # Close the figure to free up memory
    plt.close(fig)
    
    # Update the progress bar
    pbar.update(i + 1)

# Finish the progress bar
pbar.finish()


100% |########################################################################|
